In [26]:
import pandas as pd
# read user file
user_cols =['user_id','age','gender','occupation','zip_code']
users = pd.read_csv('ml-100k/ml-100k/u.user', sep ='|', names =user_cols)
print('Users shape', users.shape[0])
print('users: \n', users.head())
##Reading ratings file:
r_cols =['user_id','movie_id','rating','time']
rating_base = pd.read_csv('ml-100k/ml-100k/ua.base', sep ='\t', names =r_cols)
rating_test = pd.read_csv('ml-100k/ml-100k/ua.test', sep ='\t', names =r_cols)
print('rating_base: \n', rating_base.head())
rate_train = rating_base.as_matrix()
rate_test  = rating_test.as_matrix()
print(rate_train)
print('rate_train :', rate_train.shape[0])
print('rate_test :', rate_test.shape[0])

Users shape 943
users: 
    user_id  age gender  occupation zip_code
0        1   24      M  technician    85711
1        2   53      F       other    94043
2        3   23      M      writer    32067
3        4   24      M  technician    43537
4        5   33      F       other    15213
rating_base: 
    user_id  movie_id  rating       time
0        1         1       5  874965758
1        1         2       3  876893171
2        1         3       4  878542960
3        1         4       3  876893119
4        1         5       3  889751712
[[        1         1         5 874965758]
 [        1         2         3 876893171]
 [        1         3         4 878542960]
 ...
 [      943      1188         3 888640250]
 [      943      1228         3 888640275]
 [      943      1330         3 888692465]]
rate_train : 90570
rate_test : 9430


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if sys.path[0] == '':
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  del sys.path[0]


# Xây dựng item profile

In [79]:
# read file item
item_cols =['movie_id','movie_title','release_date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure',
 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy',
 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv('ml-100k/ml-100k/u.item', sep ='|', names =item_cols, encoding ='latin-1')
print('shape items: ', items.shape[0])
print('items :\n', items.head())
# Vì ta quan tâm đến 19 thể loại của mỗi items
X0 = items.as_matrix()
X_train_counts = X0 [:,-19:] #-19: -1
print(X_train_counts)

# Xây dựng các feature vector bằng TF-IDF
from sklearn.feature_extraction.text import TfidfTransformer
transformer = TfidfTransformer(smooth_idf = True, norm ='l2')
tfidf = transformer.fit_transform(X_train_counts.tolist()).toarray()
print('tfidf: ',tfidf) # Sau bước này : mỗi hàng của tfidf tương ứng với feature vector của một bộ phim
print('tfidf shape', tfidf.shape)

shape items:  1682
items :
    movie_id        movie_title release_date  video release date  \
0         1   Toy Story (1995)  01-Jan-1995                 NaN   
1         2   GoldenEye (1995)  01-Jan-1995                 NaN   
2         3  Four Rooms (1995)  01-Jan-1995                 NaN   
3         4  Get Shorty (1995)  01-Jan-1995                 NaN   
4         5     Copycat (1995)  01-Jan-1995                 NaN   

                                            IMDb URL  unknown  Action  \
0  http://us.imdb.com/M/title-exact?Toy%20Story%2...        0       0   
1  http://us.imdb.com/M/title-exact?GoldenEye%20(...        0       1   
2  http://us.imdb.com/M/title-exact?Four%20Rooms%...        0       0   
3  http://us.imdb.com/M/title-exact?Get%20Shorty%...        0       1   
4  http://us.imdb.com/M/title-exact?Copycat%20(1995)        0       0   

   Adventure  Animation  Children's   ...     Fantasy  Film-Noir  Horror  \
0          0          1           1   ...           0 

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if __name__ == '__main__':


Tiếp theo, với mỗi user, chúng ta cần đi tìm những bộ phim nào mà user đó đã rated, và giá trị của các rating đó.

In [47]:
import numpy as np
def get_items_rated_by_user(rate_matrix, user_id):
    """
    in each line of rate_matrix, we have infor: user_id, item_id, rating (scores), time_stamp
    we care about the first three values
    return (item_ids, scores) rated by user user_id
    """
    y = rate_matrix[:,0] # all user
    # item indices rated by user_id
    # we need to +1 to user_id since in the rate_matrix, id starts from 1 
    # while index in python starts from 0
    ids = np.where(y == user_id +1)[0]
    item_ids = rate_matrix[ids,1] -1 # index  start from 0
    score = rate_matrix[ids,2] #rating
    return (item_ids, score)

# Tìm mô hình cho mỗi user
Sử dụng thư viện Ridge Regression

In [61]:
from sklearn import linear_model
from sklearn.linear_model import Ridge

d = tfidf.shape[1] # dimension
print(d)
W = np.zeros((d,users.shape[0]))
print('W:', W.shape)
b = np.zeros((1,users.shape[0]))
print('b: ',b.shape)
n_users = users.shape[0]
print(n_users) # 943

for n in range(n_users):
    item_id, score = get_items_rated_by_user(rate_train, n)
    clf = Ridge(fit_intercept = True, alpha =0.01)
    Xhat = tfidf[item_id,:] # lấy tất cả feature vector of 1 items( movie)
    clf.fit(Xhat, score)
    
    W[:, n] = clf.coef_
    b[0, n] = clf.intercept_

# dự đoán rating cho mỗi items
Yhat =tfidf.dot(W) + b
print('predict: ', Yhat)

19
W: (19, 943)
b:  (1, 943)
943
predict:  [[2.91838332 3.99617751 1.90305533 ... 5.03731383 4.77954355 3.16575325]
 [2.80263057 3.43501556 3.29904303 ... 1.09956555 4.75937828 3.69972213]
 [3.4830386  1.56190995 1.26948894 ... 7.35604832 4.18870316 3.80591695]
 ...
 [4.12679785 4.09647029 3.02107854 ... 9.53834396 4.64249022 3.27349356]
 [3.56723527 3.45068283 2.92544446 ... 3.68357866 3.6551852  2.75740298]
 [4.13381936 3.80361821 3.09903449 ... 9.53834396 4.41870642 3.93041386]]


In [72]:
# Dưới đây là 1 ví dụ với user =10
n =10
item_id, score = get_items_rated_by_user(rate_test, n)
Yhat[item_id, n]
print(' User 10 - movie_id (movie):', item_id)
print(' User 10 - rating true:', score)
print('Predict rating: ', Yhat[item_id, n])


 User 10 - movie_id (movie): [ 37 109 110 226 424 557 722 724 731 739]
 User 10 - rating true: [3 3 4 3 4 3 5 3 3 4]
Predict rating:  [3.18348495 3.1328256  3.42107302 3.091764   3.34744746 5.20450987
 4.01304057 3.34744746 3.42107302 3.71623166]


# Đánh giá mô hình
Để đánh giá mô hình tìm được, chúng ta sẽ sử dụng Root Mean Squared Error (RMSE), tức căn bậc hai của trung bình cộng bình phương của lỗi. Lỗi được tính là hiệu của true rating và predicted rating:

In [78]:
def evaluate( Yhat, rate, W,b):
    se =0
    cnt =0
    for n in range(n_users):
        item_id, score_truth = get_items_rated_by_user(rate,n)
        score_pred = Yhat[item_id, n]
        e = score_truth - score_pred
        se +=(e*e).sum(axis =0)
        cnt +=e.size
    
    return np.sqrt(se/cnt)  
print('Evaluate RMSE of training: ',evaluate(Yhat, rate_train, W, b))
print('Evaluate RMSE of test: ',evaluate(Yhat, rate_test, W, b))

Evaluate RMSE of training:  0.9089804562826721
Evaluate RMSE of test:  1.2703282700393035
